In [2]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [1]:
import wikipedia

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [3]:
import random

In [1]:
import csv

In [5]:
language = "ru"
wikipedia.set_lang(language)

In [6]:
def parentheses(text):
    count = 0
    for i in text:
        if i == "(":
            count += 1
        elif i == ")":
            count -= 1
        if count < 0:
            return False
    return count == 0

In [7]:
def elka_quotes(text):
    count = 0
    for i in text:
        if i == "«":
            count += 1
        elif i == "»":
            count -= 1
        if count < 0:
            return False
    return count == 0

In [8]:
def quotation(text):
    count_single_quotes = 0
    count_double_quotes = 0
    for i in text:
        if i == '\'':
            count_single_quotes += 1
        if i == '\"':
            count_double_quotes += 1
    return count_single_quotes % 2 == 0 and count_double_quotes % 2 == 0

In [9]:
def russian(text):
    is_russian = True
    allowed = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя-.,/' # NB! предл с любыми цифрами не попадут в выборку
    for word in text.split():
        clear_word = word.strip('\'\"\/«».,?!#№$;:%&()-+[]`').lower()
        if not set(clear_word).issubset(allowed):
            is_russian = False
    return is_russian

In [10]:
# ф-я принимает текст из википедии и возвращает список релевантных предложений
def article_filter(text):
    stop_symb_first = (' ', '=', '.', '-', '–', '—', '(')
    stop_symb_end = ('тыс.', 'млн.', ':', ';')
    adequate_sents = []
    
    sentences = sent_tokenize(text)
    for sent in sentences:
        true_sents = sent.split('\n')
        for tsent in true_sents: # когда доходим до данных разделов, перестаем искать предложения
            if tsent == '== Примечания ==' or tsent == '== Литература ==' or tsent == '== Ссылки ==':
                return adequate_sents
            words = len(tsent.split())
            
            if (words > 3 and words < 15 and # убираем длинные и короткие предложения
                not tsent.startswith(stop_symb_first) and # убираем куски, которые начинаются не как типичное предл
                not tsent.endswith(stop_symb_end) and # убираем куски, которые оканчиваются не как типичное предложение
                tsent[0].isupper() and # убираем отрывки, начинающиеся с маленьких букв
                parentheses(tsent) and # убираем предложения с несовпадением скобок
                elka_quotes(tsent) and quotation(tsent) and # убираем предложения с несовпадением кавычек
                russian(tsent)): # убираем предложения со словами на других языках
                
                adequate_sents.append(tsent)
                
    return adequate_sents

In [11]:
article_filter(wikipedia.page("Испания").content)

['Член Европейского союза и НАТО.',
 'Омывается Атлантическим океаном на севере и западе, Средиземным морем на юге и востоке.',
 'Граничит с Францией, Андоррой и Португалией в Европе, и с Марокко в Северной Африке.',
 'В течение столетий возникало множество гипотез по поводу этимологии названия «Испания».',
 'После истощения шахт Тартесс приходит в упадок.',
 'В дальнейшем иберы были частично ассимилированы кельтами.',
 'Испанией землю иберов называли финикийцы.',
 'Иберы использовали палеоиспанское письмо, созданное ранее тартессийцами.',
 'Язык иберов не был родственен тартессийскому.',
 'Кельты Иберии прославились, как воины.',
 'После падения Западной Римской Империи королевство вестготов обретает полную независимость.',
 'Вестготы, как и другие готские племена, исповедовали арианство.',
 'Однако значительную часть населения Испании всё ещё составляли потомки иберо-римлян, исповедовавшие ортодоксальное христианство.',
 'Вестготские короли Сисебут и его сын Свинтила изгнали византий

In [14]:
# процесс обхода википедии
sentences = []
wiki_titles = [
    'рэп', 'семья', 'суеверие', 
    'тёща', 'эстонцы', 'паб', 'Россия', 'Госдеп США', 
    'русский язык', 'стереотип', 'водка', 'баран', 'школа', 
    'секс', 'порнография', 'смех', 'юмор', 'ирония', 
    'картофель'
]

current_title_idx = 0
while len(sentences) < 35000: # остановимся, когда наберем 35000 предл. (взяли с запасом)
    
    try:
        wiki_page = wikipedia.page(wiki_titles[current_title_idx])
    except wikipedia.DisambiguationError as e:
        pass
    except wikipedia.exceptions.PageError as pe:
        pass
    
    article_links = wiki_page.links # все ссылки статьи вики на др статьи вики
        
    for link in article_links:
        if link not in wiki_titles:
            wiki_titles.append(link)
             
    article = wiki_page.content # текст статьи
    
    current_title_idx += 1
    
    sentences += article_filter(article)

In [16]:
with open('not_funny_fin.txt', 'w', encoding="utf-8") as f_w:
    for s in sentences:
        f_w.write(s + '\n')

In [17]:
with open("not_funny_fin.csv", "wt", newline='', encoding="utf-8") as fp:
    writer = csv.writer(fp, delimiter=",")
    for item in sentences:
        writer.writerow([item])